In [29]:
import torch
from model_utils import set_args
import Timesnet_DANN

device = torch.device('cuda:0')
eval_mask = torch.ones(1,624).to(device)
st = torch.load('best.pth')
new_st = {key.replace("module.", ""): value for key, value in st.items()}
opt=set_args()
model = Timesnet_DANN.Model_domain(opt)
model.load_state_dict(new_st)
print("[ MODEL ] MODEL LOADED.")
model = model.to(device)
model.eval()
print(model.projection.weight.grad)

[ MODEL ] MODEL LOADED.
None


In [30]:
rand_input =  torch.randn(1, 624, 1, 2).squeeze(0).permute(1,0,2).to(device)
eval_mask = torch.ones(1,624).to(device)

In [31]:
action_list = ['idle','punch','kick','left','right','down']

In [32]:
digits = model(rand_input,eval_mask)
output = torch.argmax(digits[0],dim=1)

In [33]:
class ActivationsAndGradients:
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layer):
        self.model = model
        self.gradients = []
        self.activations = []

        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_full_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations.append(output)

    def save_gradient(self, module, grad_input, grad_output):
        # Gradients are computed in reverse order
        self.gradients = [grad_output[0]] + self.gradients

    def __call__(self, x):
        self.gradients = []
        self.activations = []
        return self.model(x)

In [34]:
digits

(tensor([[ -5.6495, -23.8264,  -0.2086,   5.5490, -11.7501,  -1.4646]],
        device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[0., 0.]], device='cuda:0', grad_fn=<ReluBackward0>))

In [35]:
loss = digits[0][0][5]
loss.backward(retain_graph=True)

In [41]:
print(model.domain_classifier.d_fc1.weight.grad)

None


In [42]:
digits

(tensor([[ -5.6495, -23.8264,  -0.2086,   5.5490, -11.7501,  -1.4646]],
        device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[0., 0.]], device='cuda:0', grad_fn=<ReluBackward0>))